Property finder relases all the data 

<h3>Scrapping all ad pages  </h3>

<h5>When using Headless option you have to add user agent 
 because Headless mode uses its own default User-Agent if it is not given as an argument.
 However some webpages may block Headless mode User-Agent to avoid unwanted traffic.
 It may result in Access denied error while trying to open a webpage. </h5>

In [ ]:
import re
from collections import deque
import joblib
from math import ceil
from time import sleep
from itertools import product
from random import sample
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from requests import get
import csv
#Get all Home pages links 


page_links=[]
for i in range(1,3090):
    page_links.append(f'https:#www.propertyfinder.eg/en/search?c=1&ob=mr&page={i}&t=1')

Options = ChromeOptions()
prefs = {'profile.managed_default_content_settings.images': 2}
Options.add_experimental_option('prefs', prefs)
Options.add_argument('--ignore-certificate-errors')
Options.add_argument('--incognito')
Options.add_argument('--headless')
user_agent =  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
Options.add_argument(f'user-agent={user_agent}')
chrome_installer = Service(ChromeDriverManager().install())
chrome = Chrome(service = chrome_installer, options  = Options)
ads_pages=[]
# Send a GET request to the webpage
total_ads_scrapped=0
for i in page_links:
    chrome.get(i)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(chrome.page_source, 'html.parser')
    # Find and print all links on the page
    links = soup.find_all('a',class_ = 'card__link')
    for link in links:
        ads_pages.append(link)
print(f'Total Ads Scrapped:{len(ads_pages)}')        
# Open a new CSV file for writing
with open('Property_finder_ad_pages', 'w') as f:
    # Create a CSV writer object
    writer = csv.writer(f)
    # Write each link to a new row in the file
    for link in ads_pages:
        writer.writerow([link])
chrome.quit()        


<h2>Optimized Code</h2>

In [ ]:
import csv
from math import ceil
from random import sample
from time import sleep

import requests
from bs4 import BeautifulSoup
from requests import get
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

# Get all Home pages links
page_links = [f'https://www.propertyfinder.eg/en/search?c=1&ob=mr&page={i}&t=1' for i in range(1, 3090)]

options = Options()
prefs = {'profile.managed_default_content_settings.images': 2}
options.add_experimental_option('prefs', prefs)
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument("--disable-gpu");
options.add_argument("--no-sandbox");
options.add_argument("--disable-extensions");
options.add_argument("--dns-prefetch-disable");
options.add_argument("start-maximized"); # https://stackoverflow.com/a/26283818/1689770
options.add_argument("enable-automation"); # https://stackoverflow.com/a/43840128/1689770
options.add_argument("--disable-dev-shm-usage"); #https://stackoverflow.com/a/50725918/1689770
options.add_argument("--disable-browser-side-navigation"); #https://stackoverflow.com/a/49123152/1689770
options.add_argument("--disable-infobars"); #https://stackoverflow.com/a/43840128/1689770
#! eager : This strategy causes Selenium to wait for the DOMContentLoaded event (html content downloaded and parsed only).
#!we will use eager because the items we scrapping is loading fast 
options.page_load_strategy = 'eager'

options.add_argument('--headless')
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
options.add_argument(f'user-agent={user_agent}')

chrome = Chrome(ChromeDriverManager().install(), options=options)

ads_pages = []
total_ads_scrapped = 0

# Send a GET request to the webpage
for i in page_links:
    try:
        chrome.get(i)
    except TimeoutException as e:
        print(f"Failed to load page: {i}. Error: {e}")
        continue

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(chrome.page_source, 'html.parser')

    # Find and print all links on the page
    links = soup.select('a.card__link')
    ads_pages.extend(links)

print(f'Total Ads Scrapped:{len(ads_pages)}')

# Open a new CSV file for writing
with open('Property_finder_ad_pages.csv', 'wb') as f:
    # Create a CSV writer object
    writer = csv.writer(f)
    # Write each link to a new row in the file
    writer.writerows([str(link)] for link in ads_pages)

chrome.quit()


In [ ]:
print(f'Total Ads Scrapped:{len(ads_pages)}')

# Open a new CSV file for writing
with open('Property_finder_ad_pages_final.csv', 'w') as f:
    # Create a CSV writer object
    writer = csv.writer(f)
    # Write each link to a new row in the file
    writer.writerows([[link] for link in ads_pages])

chrome.quit()


Set Options

In [3]:
import csv
from math import ceil
from random import sample
from time import sleep
import requests
from bs4 import BeautifulSoup
from requests import get
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import re
import pandas as pd 
import numpy as np 
from collections import deque
import joblib
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'

def set_options(*args):
    options = Options()
    prefs = {'profile.managed_default_content_settings.images': 2}
    options.add_experimental_option('prefs', prefs)
    
    for arg in args:
        options.add_argument(arg)
        
    #! eager : This strategy causes Selenium to wait for the DOMContentLoaded event (html content downloaded and parsed only).
    #!we will use eager because the items we scrapping is loading fast 
    options.page_load_strategy = 'eager'
    return options

Scrapping all properties 

In [4]:
# Get all Home pages links
def scrapp_all_properties():
    page_links = [f'https://www.propertyfinder.eg/en/buy/properties-for-sale.html?page={i}' for i in range(1, 4000)]

    chrome = Chrome(ChromeDriverManager().install(), options=set_options('--headless','--ignore-certificate-errors','--incognito',"--disable-gpu","--no-sandbox","--disable-extensions","--dns-prefetch-disable","start-maximized","enable-automation","--disable-dev-shm-usage","--disable-browser-side-navigation","--disable-infobars",f'user-agent={user_agent}'))

    ads_pages = []
    total_ads_scrapped = 0

    # Send a GET request to the webpage
    for i in page_links:
        try:
            chrome.get(i)
        except TimeoutException as e:
            print(f"Failed to load page: {i}. Error: {e}")
            continue
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(chrome.page_source, 'html.parser')
        # Find and print all links on the page
        links = soup.select('a.card__link')
        ads_pages.extend(links)
        print(f'links visited:{chrome.current_url.split("=")[1]}')
        break;

    print(f'Total Ads Scrapped:{len(ads_pages)}')
    #Convert it to set to remove duplicated links
    set(ads_pages)
    home_page_all_ads=set(ads_pages)
    # joblib.dump(ads_pages, 'state2.pkl')
    # This line is creating two empty lists, 'href' and 'title', to which we will append values in the loop below.
    # This line iterates over each element 'x' in the list 'ads_pages'. 
    #!For each element x it extracts the 'href' attribute and appends it to the 'href' list, and also extracts the 'title' attribute and appends it to the 'title' list. 
    #!The results are stored as tuples inside the respective lists.


    # Create an empty deque
    href_deque = deque()

    # Use list comprehension to append items to deque
    ([(href_deque.appendleft(x.get('href'))) for x in ads_pages])
    joblib.dump(href_deque, 'state2.pkl')



    # Open a new CSV file for writing
    # with open('Property_finder_ad_pages.csv', 'wb') as f:
    #     # Create a CSV writer object
    #     writer = csv.writer(f)
    #     # Write each link to a new row in the file
    #     writer.writerows([str(link)] for link in ads_pages)
    chrome.quit()
    
scrapp_all_properties()

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:07<00:00, 938kB/s] 
C:\Users\amigo\AppData\Local\Temp\ipykernel_11548\95112749.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = Chrome(ChromeDriverManager().install(), options=set_options('--headless','--ignore-certificate-errors','--incognito',"--disable-gpu","--no-sandbox","--disable-extensions","--dns-prefetch-disable","start-maximized","enable-automation","--disable-dev-shm-usage","--disable-browser-side-navigation","--disable-infobars",f'user-agent={user_agent}'))


links visited:1
Total Ads Scrapped:25


In [9]:
# joblib.dump(ads_pages, 'state2.pkl')


href_deque = joblib.load('state2.pkl')
href_deque

deque(['https://www.propertyfinder.eg/en/plp/buy/chalet-for-sale-north-coast-sidi-abdel-rahman-marassi-3846663.html',
       'https://www.propertyfinder.eg/en/plp/buy/villa-for-sale-cairo-new-cairo-city-ext-north-inves-area-dyar-3909035.html',
       'https://www.propertyfinder.eg/en/plp/buy/duplex-for-sale-cairo-new-cairo-city-the-5th-settlement-mostakbal-city-compounds-il-bosco-city-3848318.html',
       'https://www.propertyfinder.eg/en/plp/buy/apartment-for-sale-cairo-new-cairo-city-the-5th-settlement-5th-settlement-compounds-palm-hills-new-cairo-3855088.html',
       'https://www.propertyfinder.eg/en/plp/buy/villa-for-sale-cairo-mokattam-uptown-cairo-levana-3867647.html',
       'https://www.propertyfinder.eg/en/plp/buy/villa-for-sale-giza-sheikh-zayed-city-sheikh-zayed-compounds-royal-city-3883907.html',
       'https://www.propertyfinder.eg/en/plp/buy/townhouse-for-sale-suez-al-ain-al-sokhna-azha-3865761.html',
       'https://www.propertyfinder.eg/en/plp/buy/apartment-for-sale-

In [ ]:
print(f'Total Ads Scrapped:{len(ads_pages)}')

# Open a new CSV file for writing
with open('All_properties_property_finder_ad_pages.csv', 'wb') as f:
    # Create a CSV writer object
    writer = csv.writer(f)
    # Write each link to a new row in the file
    writer.writerows([str(link)] for link in ads_pages)

chrome.quit()

In [10]:
href_deque[0]

'https://www.propertyfinder.eg/en/plp/buy/chalet-for-sale-north-coast-sidi-abdel-rahman-marassi-3846663.html'

Static Scrapp Ad Pages

In [11]:
# Create an empty list to store the amenities
#TODO add joblib
#TODO add deqeue
#TODO save all variables into dictionary to dump it to pickle with joblib 

amenities_list = []
ads_list_dic=[]
ad={'url':href_deque[4]}

chrome = Chrome(ChromeDriverManager().install(), options=set_options('--headless','--ignore-certificate-errors','--incognito',"--disable-gpu","--no-sandbox","--disable-extensions","--dns-prefetch-disable","start-maximized","enable-automation","--disable-dev-shm-usage","--disable-browser-side-navigation","--disable-infobars",f'user-agent={user_agent}'))


chrome.get(href_deque[4])
            #TODO make this dynamic 
soup = BeautifulSoup(chrome.page_source, 'html.parser')
# Find and print subtitle of the page
#!####################################################
ad['title'] = soup.find('h1',class_ = 'property-page__title').text.strip()
ad['sub_title'] = soup.find('h2',class_ = 'property-page__sub-title').text.strip()
li = soup.find_all('div',class_ = 'property-facts__value')
ad['property_type']=soup.find_all('div',class_ = 'property-facts__value')[0].text.strip()
#Get all text
property_size_temp=soup.find_all('div',class_ = 'property-facts__value')[1].text.strip()
#Extract SQM size
ad['property_size']=re.sub(r'\D*(\d+)\D*',r'\1', li[1].text.strip()[15:]).strip()
ad['bedrooms']=soup.find_all('div',class_ = 'property-facts__value')[2].text.replace('\n','').replace('\t', '').strip()
ad['bathrooms']=soup.find_all('div',class_ = 'property-facts__value')[3].text.strip()
ad['agent_name']=soup.find('h4',class_ = 'property-agent__name').text.strip()
ad['agent_company']=soup.find('div',class_ = 'property-agent__position-broker-name').text.strip()
ad['agent_number'] = soup.find('a', class_='property-contact__primary-button').get('href')[4:]
ad['description']=soup.find('div',class_ = 'property-description__text-trim').text.replace('\n','').strip()
ad['listed_date']=soup.find_all('div',class_ = 'property-page__legal-list-content')[1].text.strip()
#! Ameneties 
# Get all the divs with class 'property-amenities__list' from a BeautifulSoup object called soup
amenities_all_divs = soup.find_all('div', class_='property-amenities__list')
# Iterate over each div in the amenities_all_divs list and add its text content (with leading/trailing whitespace removed) to the amenities_list
[amenities_list.append(x.text.strip()) for x in amenities_all_divs]
# Join the elements of the amenities_list together into a single string, separated by semicolons (;), and assign the result to the amenities variable
ad['amenities'] = ";".join(amenities_list)

#Add Dictionary to list 
ads_list_dic.append(ad)

#pop first left link 


C:\Users\amigo\AppData\Local\Temp\ipykernel_11548\1708869032.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = Chrome(ChromeDriverManager().install(), options=set_options('--headless','--ignore-certificate-errors','--incognito',"--disable-gpu","--no-sandbox","--disable-extensions","--dns-prefetch-disable","start-maximized","enable-automation","--disable-dev-shm-usage","--disable-browser-side-navigation","--disable-infobars",f'user-agent={user_agent}'))


In [12]:
href_deque

deque(['https://www.propertyfinder.eg/en/plp/buy/chalet-for-sale-north-coast-sidi-abdel-rahman-marassi-3846663.html',
       'https://www.propertyfinder.eg/en/plp/buy/villa-for-sale-cairo-new-cairo-city-ext-north-inves-area-dyar-3909035.html',
       'https://www.propertyfinder.eg/en/plp/buy/duplex-for-sale-cairo-new-cairo-city-the-5th-settlement-mostakbal-city-compounds-il-bosco-city-3848318.html',
       'https://www.propertyfinder.eg/en/plp/buy/apartment-for-sale-cairo-new-cairo-city-the-5th-settlement-5th-settlement-compounds-palm-hills-new-cairo-3855088.html',
       'https://www.propertyfinder.eg/en/plp/buy/villa-for-sale-cairo-mokattam-uptown-cairo-levana-3867647.html',
       'https://www.propertyfinder.eg/en/plp/buy/villa-for-sale-giza-sheikh-zayed-city-sheikh-zayed-compounds-royal-city-3883907.html',
       'https://www.propertyfinder.eg/en/plp/buy/townhouse-for-sale-suez-al-ain-al-sokhna-azha-3865761.html',
       'https://www.propertyfinder.eg/en/plp/buy/apartment-for-sale-

<h4>Scrapp Ad Pages </h4>

In [19]:
# Create an empty list to store the amenities
#TODO add joblib
#TODO add deqeue
#TODO save all variables into dictionary to dump it to pickle with joblib 

amenities_list = []
ads_list_dic=[]
ad={'url':href_deque[-1]}

chrome = Chrome(ChromeDriverManager().install(), options=set_options('--headless','--ignore-certificate-errors','--incognito',"--disable-gpu","--no-sandbox","--disable-extensions","--dns-prefetch-disable","start-maximized","enable-automation","--disable-dev-shm-usage","--disable-browser-side-navigation","--disable-infobars",f'user-agent={user_agent}'))
try:    
    ads_list_dic, href_deque = jobli0b.load('state.pkl')
except:
    #FIXME not logic to scrapp all again 
    scrapp_all_properties()
    # ads = joblib.load('state2.pkl')

    
finally:
    
    while href_deque:
        #it will pop from the left (newest added) to the oldest 
        href_deque.popleft()
        current_url = href_deque.popleft()
        print(current_url)
        scraped_urls = {x['url'] for x in ads_list_dic}
        
        if current_url not in scraped_urls:
            print(len(ads), current_url)
        
        
        while True:
                try:
                    chrome.get(current_url)
                    #sleep(2.0)
                    break
                except TimeoutException:
                    print('timeout detected... cooling off...')
                    sleep(2.0)
                    # chrome.get(href_deque[0])
                    #TODO make this dynamic 

        soup = BeautifulSoup(chrome.page_source, 'html.parser')
        # Find and print subtitle of the page
        #!####################################################
        ad['title'] = soup.find('h1',class_ = 'property-page__title').text.strip()
        ad['sub_title'] = soup.find('h2',class_ = 'property-page__sub-title').text.strip()
        li = soup.find_all('div',class_ = 'property-facts__value')
        ad['property_type']=soup.find_all('div',class_ = 'property-facts__value')[0].text.strip()
        #Get all text
        property_size_temp=soup.find_all('div',class_ = 'property-facts__value')[1].text.strip()
        #Extract SQM size
        ad['property_size']=re.sub(r'\D*(\d+)\D*',r'\1', li[1].text.strip()[15:]).strip()
        ad['bedrooms']=soup.find_all('div',class_ = 'property-facts__value')[2].text.replace('\n','').replace('\t', '').strip()
        ad['bathrooms']=soup.find_all('div',class_ = 'property-facts__value')[3].text.strip()
        ad['agent_name']=soup.find('h4',class_ = 'property-agent__name').text.strip()
        ad['agent_company']=soup.find('div',class_ = 'property-agent__position-broker-name').text.strip()
        ad['agent_number'] = soup.find('a', class_='property-contact__primary-button').get('href')[4:]
        ad['description']=soup.find('div',class_ = 'property-description__text-trim').text.replace('\n','').strip()
        ad['listed_date']=soup.find_all('div',class_ = 'property-page__legal-list-content')[1].text.strip()
        #! Ameneties 
        # Get all the divs with class 'property-amenities__list' from a BeautifulSoup object called soup
        amenities_all_divs = soup.find_all('div', class_='property-amenities__list')
        # Iterate over each div in the amenities_all_divs list and add its text content (with leading/trailing whitespace removed) to the amenities_list
        [amenities_list.append(x.text.strip()) for x in amenities_all_divs]
        # Join the elements of the amenities_list together into a single string, separated by semicolons (;), and assign the result to the amenities variable
        ad['amenities'] = ";".join(amenities_list)

        #Add Dictionary to list 
        ads_list_dic.append(ad)
        state = ads_list_dic, href_deque
        joblib.dump(state, 'scrap.pkl')
        #pop first left link 


IndexError: deque index out of range

In [41]:
import joblib
state = ads_list_dic, href_deque
joblib.dump(state, 'state.pkl')

['state.pkl']

In [44]:
ads_list_dic, href_deque = joblib.load('state.pkl')


In [14]:
href_deque

deque([])

In [33]:
while href_deque:
    #it will pop from the left (newest added) to the oldest 
    href_deque.popleft()
    current_url = href_deque.popleft()
    print(current_url)
    scraped_urls = {x['url'] for x in ads_list_dic}
    
    if current_url not in scraped_urls:
        print(len(ads_list_dic), current_url)
    
    
    while True:
            try:
                chrome.get(current_url)
                #sleep(2.0)
                break
            except TimeoutException:
                print('timeout detected... cooling off...')
                sleep(2.0)
                # chrome.get(href_deque[0])
                #TODO make this dynamic 

https://www.propertyfinder.eg/en/plp/buy/apartment-for-sale-cairo-new-cairo-city-the-5th-settlement-5th-settlement-compounds-palm-hills-new-cairo-3882856.html
2 https://www.propertyfinder.eg/en/plp/buy/apartment-for-sale-cairo-new-cairo-city-the-5th-settlement-5th-settlement-compounds-palm-hills-new-cairo-3882856.html
https://www.propertyfinder.eg/en/plp/buy/twin-house-for-sale-cairo-new-cairo-city-the-5th-settlement-5th-settlement-compounds-layan-residence-3849963.html
2 https://www.propertyfinder.eg/en/plp/buy/twin-house-for-sale-cairo-new-cairo-city-the-5th-settlement-5th-settlement-compounds-layan-residence-3849963.html
https://www.propertyfinder.eg/en/plp/buy/ivilla-for-sale-giza-6-october-city-6th-district-mountain-view-october-park-3829754.html
2 https://www.propertyfinder.eg/en/plp/buy/ivilla-for-sale-giza-6-october-city-6th-district-mountain-view-october-park-3829754.html
https://www.propertyfinder.eg/en/plp/buy/apartment-for-sale-cairo-new-cairo-city-the-5th-settlement-5th-se

KeyboardInterrupt: 

In [15]:

ads_list_dic.append(ad)

In [15]:
import pandas as pd
df=pd.DataFrame(ads_list_dic)

In [16]:
df

,url,title,sub_title,property_type,property_size,bedrooms,bathrooms,agent_name,agent_company,agent_number,description,listed_date,amenities
0,https://www.propertyfinder.eg/en/plp/buy/townh...,Sea view - installment -Soma Breeze,"Chalet for Sale in Soma Breeze, Soma Bay",Chalet,70,1,1,Fatma Al Moug,Remax The Address,+201009040536,Chalet for sale in Soma BreezeSea viewGround F...,1 month ago,Furnished;Kitchen Appliances;Central A/C;Priva...
1,https://www.propertyfinder.eg/en/plp/buy/townh...,Sea view - installment -Soma Breeze,"Chalet for Sale in Soma Breeze, Soma Bay",Chalet,70,1,1,Fatma Al Moug,Remax The Address,+201009040536,Chalet for sale in Soma BreezeSea viewGround F...,1 month ago,Furnished;Kitchen Appliances;Central A/C;Priva...
2,https://www.propertyfinder.eg/en/plp/buy/townh...,Sea view - installment -Soma Breeze,"Chalet for Sale in Soma Breeze, Soma Bay",Chalet,70,1,1,Fatma Al Moug,Remax The Address,+201009040536,Chalet for sale in Soma BreezeSea viewGround F...,1 month ago,Furnished;Kitchen Appliances;Central A/C;Priva...
3,https://www.propertyfinder.eg/en/plp/buy/townh...,Sea view - installment -Soma Breeze,"Chalet for Sale in Soma Breeze, Soma Bay",Chalet,70,1,1,Fatma Al Moug,Remax The Address,+201009040536,Chalet for sale in Soma BreezeSea viewGround F...,1 month ago,Furnished;Kitchen Appliances;Central A/C;Priva...
4,https://www.propertyfinder.eg/en/plp/buy/townh...,Sea view - installment -Soma Breeze,"Chalet for Sale in Soma Breeze, Soma Bay",Chalet,70,1,1,Fatma Al Moug,Remax The Address,+201009040536,Chalet for sale in Soma BreezeSea viewGround F...,1 month ago,Furnished;Kitchen Appliances;Central A/C;Priva...
5,https://www.propertyfinder.eg/en/plp/buy/townh...,Sea view - installment -Soma Breeze,"Chalet for Sale in Soma Breeze, Soma Bay",Chalet,70,1,1,Fatma Al Moug,Remax The Address,+201009040536,Chalet for sale in Soma BreezeSea viewGround F...,1 month ago,Furnished;Kitchen Appliances;Central A/C;Priva...
6,https://www.propertyfinder.eg/en/plp/buy/townh...,Sea view - installment -Soma Breeze,"Chalet for Sale in Soma Breeze, Soma Bay",Chalet,70,1,1,Fatma Al Moug,Remax The Address,+201009040536,Chalet for sale in Soma BreezeSea viewGround F...,1 month ago,Furnished;Kitchen Appliances;Central A/C;Priva...
7,https://www.propertyfinder.eg/en/plp/buy/townh...,Sea view - installment -Soma Breeze,"Chalet for Sale in Soma Breeze, Soma Bay",Chalet,70,1,1,Fatma Al Moug,Remax The Address,+201009040536,Chalet for sale in Soma BreezeSea viewGround F...,1 month ago,Furnished;Kitchen Appliances;Central A/C;Priva...
8,https://www.propertyfinder.eg/en/plp/buy/townh...,Sea view - installment -Soma Breeze,"Chalet for Sale in Soma Breeze, Soma Bay",Chalet,70,1,1,Fatma Al Moug,Remax The Address,+201009040536,Chalet for sale in Soma BreezeSea viewGround F...,1 month ago,Furnished;Kitchen Appliances;Central A/C;Priva...
9,https://www.propertyfinder.eg/en/plp/buy/townh...,Sea view - installment -Soma Breeze,"Chalet for Sale in Soma Breeze, Soma Bay",Chalet,70,1,1,Fatma Al Moug,Remax The Address,+201009040536,Chalet for sale in Soma BreezeSea viewGround F...,1 month ago,Furnished;Kitchen Appliances;Central A/C;Priva...
